# Unit 4: Data Modeling Basics

## Contents

* [Getting Started](#Getting-Started)
* [Linear Models](#LinearModels)
    * [Concept: Linear Regression](#Concept-LinearRegression)
    * [Simple Linear Regression](#SimpleLinearRegression)
    * [Multiple Linear Regression](#MultipleLinearRegression)
    * [Linear-Like Regression](#Linear-Like-Regression)
* [Logistic Regression](#Logistic-Regression)
    * [An Example](#An-Example)
    * [Home Data](#Home-Data)
* [Exercise Answers](#Exercises)
* [Next Steps](#Next-Steps)
* [Resources and Further Reading](#Resources-and-Further-Reading)
* [Notes](#Notes)

### Exercises

[1](#Exercise-1), [2](#Exercise-2), [3](#Exercise-3), [4](#Exercise-4), [5](#Exercise-5), [6](#Exercise-6), [7](#Exercise-7), [8](#Exercise-8)

## Getting Started

In previous units, we worked on loading, cleaning, and exploring data.  While working with the data, we noted that certain relationships appeared to exist between columns/variables.  While plots allowed us to make claims like "x increases as y decreases",  we didn't try to model that relationship mathematically nor did we try to determine the quality of that model.  

In this unit,we'll look at creating models for the relationships in our data; specifically, we'll look at linear models for numerical data and logistic models for categorical data.

To create and explore these models, we'll use the [StatsModels](https://www.statsmodels.org/stable/index.html) library. To install it, we'll use `!pip`.

If using [Anaconda](https://www.anaconda.com/download) and the following pip command fails, open the Anaconda prompt on your computer and run the following

```
conda install --yes statsmodels
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
!{sys.executable} -m pip install statsmodels

We'll work with plots in this unit.  To ensure they are displayed in the notebook itself, we'll need to use the `%matplotlib inline` command.

In [ ]:
%matplotlib inline

In addition to the Seaborn and pandas libraries, we'll make explicit use of libraries on which they depend.

- [matplotlib.pyplot](https://matplotlib.org/api/pyplot_api.html): a collection of plotting functions with [MATLAB](https://www.mathworks.com/products/matlab.html)-like syntax
- [numpy](http://www.numpy.org/): scientific computing library

We'll import these and StatsModels with names that follow standard convention.

We also set the figure size for plots and a marker size for outliers in box plots.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

sns.set(rc={'figure.figsize':(10,10), "lines.markeredgewidth": 0.5 })

<a name="LinearModels"></a>
## Linear Models

<hr>

<a name="Concept-LinearRegression"></a>
#### Concept: Linear Regression

Let's look at an example of how we can calculate the coefficients of a linear equation that models some data. We'll start with an example based on the [StatsModels documentation](http://www.statsmodels.org/stable/examples/notebooks/generated/ols.html).

First, we generate our "observed" data.  To do this, we'll use the Numpy [*linspace()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) function to generate 100 evenly-spaced values between 0 and 10; these will correspond to values that will be used for the independent variable.  Next, we'll use the [*normal()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.normal.html) function from NumPy's [*random*](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html) submodule to draw 100 samples from a normal distribution - this will simulate errors in our data.

In [ ]:
nsample = 100
x = np.linspace(0, 10, nsample)
e = np.random.normal(size=nsample)

display(x[:10])

Looking at the first 10 values of `x`, we can see that they are stored in an [array](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.array.html).

Next, we calculate our "observed" values as a combination of the independent variable, a constant, and some error.

In [ ]:
y = 5 * x + 3 + e

Let's plot the values of `x` and `y`.  We'll create a scatter plot but use a different approach to do this.  

First, we create [figure](https://matplotlib.org/api/_as_gen/matplotlib.figure.Figure.html#matplotlib.figure.Figure) and [axes](https://matplotlib.org/api/axes_api.html#matplotlib.axes.Axes) objects using the pyplot [*subplots()*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplots.html) function; this is useful when we want to plot items from different sources together (as we will do in a bit). We use the axes' [*plot()*](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.plot.html#matplotlib.axes.Axes.plot) method to plot the coordinate pairs from `x` and `y`; the third argument indicates that we'd like to use blue plus-sign markers rather than draw lines from the data.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, 'b+')

As shown by the plot, the data does look linearly related.  One measure of the strength of the relationship is the [correlation coefficient](https://en.wikipedia.org/wiki/Correlation_coefficient).  Given two variables, the correlation coefficent can have a value between -1 and 1 where -1 indicates strong, negative relationship (as one variable increases, the other decreases) and 1 indicates a strong, postive relationship (as one variable increases, the other increases); a valeu of zero indicates no relationship exists.

To calculate the correlation coefficient, we can use the NumPy [*corrcoef()*](https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html) function.

In [ ]:
np.corrcoef(x, y)

The output is the correlation coefficient matrix that shows from left to right, top to bottom, correlation cofficient between the first variable and itself, the correlation cofficient between the first variable and the second, the correlation cofficient between the second variable and the first, and the correlation cofficient between the second variable and itself.  We expect that a variable will be strongly correlated with itself.  The output indicates a strong relationship between the variables.  By construction, the relationship is not only strong, it is also very linear.  To see this, we can use a linear regression.

In the two-dimensional linear regression. we need to calculate the values of two coefficients: a constant and a value that will be multiplied by the value of the independent variable.  As we saw above, we can write the linear model in the form

$$
\begin{array}{c}
Y = \beta_0 + \beta_1 X
\end{array}
$$

We can write the constant, $\beta_0$ as $\beta_0 X^0$  Since any value raised to the zeroth power is one, we can write $\beta_0$ as $1 \cdot \beta_0$.  We can say that the dependent variable is a linear combination of 1 and the independent variable.  For our model, we account for this by using the StatsModels' [*add_constant()*](http://www.statsmodels.org/dev/generated/statsmodels.tools.tools.add_constant.html) function. 

In [ ]:
X = sm.add_constant(x)
display(X[:10])

Comparing `X` to `x` we now have an array of arrays where the first element in the inner arrays are `1` corresponding to the value of the independent variable for the constant term. The corresponds to the matrix $\mathbf{X}$ described above.

To compute the the linear regression, we first set up the ordinary least squares model using the StatsModels' [*OLS*](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) function and by specifying the array of values for the dependent variable and the array of values for the independent variable.  

With the model created, we calculate the regression coefficients using the model's [*fit()*](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.fit.html#statsmodels.regression.linear_model.OLS.fit) method; this method returns a [*RegressionResults*](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html) object. To dispaly information about the fit, we display the output of the result's [*summary()*](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.summary.html#statsmodels.regression.linear_model.RegressionResults.summary) method.

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
display(results.summary())

From the results, the number of observations, the coefficients, their p-values, and the value of `R-squared` are particularly of interest.  We can access these directly from `results` using the *nobs*, *params*, *pvalues*, and *rsquared* properties as well. (See StatsModels [RegressionResults](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults) for more options)

In [ ]:
print('No. of Obs.: ', results.nobs)
print('Parameters: ', results.params)
print("P-values:", results.pvalues)
print('R^2: ', results.rsquared)

The coefficients appear in the same order in which the independent variables appear in `X`.

Note that this is "close" to the equation we used to generate the data - the discrepancy is due to the error we introduced.  

Let's plot the regression line along with our data.  We can repeat the same steps as before to create the scatter plot.  We make an additional call to *plot()* and specify the y-values as the output from the results *predict()* method which returns the predicted values of the dependent variable based on the regression results.  Specifying `'r'` for the third argument to plot indicates that we would like the line to be red.  

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(x, y, 'b+')
ax.plot(x, results.predict(), 'r')

The [p-value](https://en.wikipedia.org/wiki/P-value) associated with each coefficient indicates how likely changes to the corresponding independent variable account for changes in the dependent variable.  A p-value close to zero (typically, less than 0.05) indicates that the independent variable provides a meaningful addition to the model. <sup><a href="#note-1">1</a></sup>

The `R-squared` value is also known as the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) and provides a measure of how well the regression line fits the data.  The coefficient of determination can range from 0 to 1 with 0 indicating (in some regressions, the value can be negative) how much of the variation in the dependent variable can be explained by the model - a value of 1 indicates that all variation is explained by the model and 0 indicates that the model accounts for none of the variation. In this example, some of the variation is due to the error we introduced - changing the magnitude of the error or the parameters of the distribution from which values were drawn will result in a better or worse coefficient of determination.

Assuming the fit is a good one, we can use it to predict other values not in the initial data. The simplest way to do this for individual values is by defining a function that takes an input value as a parameter and computes and returns the output based on the calculate coefficients. (In the example below, we place a `'_'` character after the function name, `predict_` to keep it distinct from the `predict` method provided by the statsmodels package.


In [ ]:
def predict_(model_results, value):
    const, coeff = model_results.params
    return const + coeff * value

print(predict_(results, 4.5))
print(predict_(results, 15))

<hr>

<a name="SimpleLinearRegression"></a>
### Simple Linear Regression

Let's look at an example based on real data. To begin, we'll reload the county auditor data we processed previously.

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/output.sqlite')

query = "SELECT * from pit_df;"
df = pd.read_sql_query(query, con=engine)
display(df.head())

Remember that in Unit 3 we had more reasonable distributions of sale prices when we did some filtering. Let's load the "sampling.py" file, and use the included function `unit3_df_redux` to repeat the data modifications we made.

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("../src/cscc_ada"))
from sampling import *

df_sub = unit3_df_redux(df)
df_sub.head()

To start, let's look at a the `yearbuilt`, `appraisedtaxablebuilding`, `saleprice`, and `area` columns in `df_sub`. First we can calculate the correleation coefficient matrix. With a DataFrame, we can use the *corr()* method to calculate this.

In [ ]:
display(df_sub[["yearbuilt", "appraisedtaxablebuilding", "saleprice", "area", "bathrooms", "bedrooms", "acreage"]].corr())

No change, so although removing those rows didn't improve our results, it didn't negatively impact them either, which is good. To get sense of these relationships visually we can use a pair plot, just like what we did in Unit 3.

<hr>

<a name="Exercise-1"></a><mark> **Exercise 1** In the cell below, create a pair plot for the  `yearbuilt`, `appraisedtaxablebuilding`, `saleprice`, `area`, `bathrooms`, `bedrooms`, and `acreage` columns in `df_avg`.
</mark>

Let's begin by  look more closely at the most correlated relationship we found which was between `area` and `bathrooms`.

<hr>

<a name="Exercise-2"></a><mark> **Exercise 2** In the cell below, create scatter plot for the `area` and `bathrooms` columns in `df_avg`.
</mark>

Once again we see that there are dwellings with areas of zero and zero bathrooms. These dwellings are unimportant for our analysis, so let's drop them and create a new subset to work with further. Additionally, let's not consider any dwelling with a sale price or appraised value above 500k dollars because there is too much noise in those properties to be useful for our simple linear regression.

In [ ]:
df_avg = df_sub[df_sub.area > 0]
df_avg = df_avg[df_avg.bathrooms > 0]
df_avg = df_avg[df_avg.saleprice < 500000]
df_avg = df_avg[df_avg.appraisedtaxablebuilding < 500000]

Now let's review the pair plots again.

In [ ]:
sns.pairplot(df_avg[columns])

<hr>

Next, lets create the least squares model and fit a line to the data.




In [ ]:
X = sm.add_constant(df_avg.appraisedtaxablebuilding)
Y = df_avg.saleprice
model = sm.OLS(Y, X)
res = model.fit()
display(res.summary())

From the `R-squared` value we can see there is a moderate linear relationship between the data; further, the p-value for the coefficient of `saleprice` indicates that changes in `displ` are likely attributed to changes in `cylinders`.  

To plot the fit line with the scatter plot generated by the DataFrame's *plot()* method we can use StatsModel's [*abline_plot()*](http://www.statsmodels.org/dev/generated/statsmodels.graphics.regressionplots.abline_plot.html) function.  First, we import the function then create a scatter plot and store the returned *axes* object.

In [ ]:
from statsmodels.graphics.regressionplots import abline_plot
axes = df_avg.plot.scatter(x="appraisedtaxablebuilding", y="saleprice", c='b')

We can can add the plot of the regression line to the plot using *abline_plot()* function, specifying the model results, the *axes*, and a color.

In [ ]:
abline_plot(model_results=res, ax=axes, color='r')

The chart above shows just how many outliers we have in our data. There are likely multiple reasons why dwellings sell for much more than they are appraised for (the left-hand side of the chart), as well as reasons for selling for far less than they are appraised for (bottom of the chart). A perfect correlation would be a 45-degree diagonal line as we saw in our example code above. With real-world data such a result is rare. For the purpose of illustrating more linear regression techniques, we are going to "cheat" and temporarily remove most of the outliers from our data. We can also use Seaborn as we did in Unit 3, and allow the `lmplot()` function to perform automatic regression fits for us by category. 

In [ ]:
df_avg_mod = df_avg.query("not (saleprice / appraisedtaxablebuilding > 3.5) "
                             "and not (appraisedtaxablebuilding / saleprice > 3.5)")
sns.lmplot(x='appraisedtaxablebuilding', y="saleprice",
           height=10, hue='condition', 
           data=df_avg_mod, fit_reg=True)

When we arbitrarily remove all of those outliers, we can see that our fit lines do begin to match the general trend that is represented by all of the dots. Let's re-run our regression and then plot against our original regression data.

In [ ]:
X2 = sm.add_constant(df_avg_mod.appraisedtaxablebuilding)
Y2 = df_avg_mod.saleprice
model2 = sm.OLS(Y2, X2)
res2 = model2.fit()
display(res2.summary())
axes2 = df_avg_mod.plot.scatter(x="appraisedtaxablebuilding", y="saleprice", c='b')

We can see that our `R-squared` value improved from 0.526 to 0.733, which means that our prediction accuracy is better. Let's look at it graphically.

In [ ]:
abline_plot(model_results=res, ax=axes, color='b')
abline_plot(model_results=res2, ax=axes, color='r')

The original regression line is colored red and the regression line calculated with outliers removed is colored red.

By removing so many outliers, what types of assumptions might we be making? If we remove all of those datapoints, how might we be over-simplifying our model? These are just some of the questions we would want to resolve before declaring our linear regression model complete and useful for making inferrances and predictions. However, for this lesson we are primarily trying to learn how to perform a linear regression and refine our results, not to build a fully functional housing prediction model.

<a name="MultipleLinearRegression"></a>
### Multiple Linear Regression

So far, we've looked at regressions in which there is one independent variable and a constant.  Often, changes in the response variable are dependent on multiple variables. For example. we expect that the sale price, `saleprice`, is dependent on both appraised value, `appraisedtaxablebuilding`, and the size of the dwelling, `area`. We can see from the scatter plots that `comb08` looks linearly dependent on `city08` and `highway08`.

In [ ]:
df_avg_mod.plot.scatter(x='appraisedtaxablebuilding', y='saleprice')
df_avg_mod.plot.scatter(x='area', y='saleprice')

We can also fit models for each of these individually - `appraisedtaxablebuilding`/`saleprice` and `area`/`saleprice`.

In [ ]:
X = sm.add_constant(df_avg_mod.appraisedtaxablebuilding)
Y = df_avg_mod.saleprice
model = sm.OLS(Y, X)
res = model.fit()
display(res.summary())

In [ ]:
X = sm.add_constant(df_avg_mod.area)
Y = df_avg_mod.saleprice
model = sm.OLS(Y, X)
res = model.fit()
display(res.summary())

Separately, the models fit the data well enough. Let's look at the model in which both `appraisedtaxablebuilding` and `area` are independent variables used to predict `saleprice`.

In [ ]:
X = sm.add_constant(df_avg_mod[['area','appraisedtaxablebuilding']])
Y = df_avg_mod.saleprice
model = sm.OLS(Y, X)
res = model.fit()
display(res.summary())

Looking at the coefficient of determination, we can see this model, which depends on both `appraisedtaxablebuilding` and `area`, fits the data a bit better than a model which depends on only one of the variables.

We can predict values using the results in a method similar to before using a a custom function that accepts values for each of the independent variables.

In [ ]:
def predict(model_results, area, appraisedtaxablebuilding):
    const, area_coeff, appraisedtaxablebuilding_coeff = model_results.params
    return const + area_coeff * area + appraisedtaxablebuilding_coeff * appraisedtaxablebuilding

print(predict(res, 2500, 150000))
print(predict(res, 4500, 300000))

For our problem, the results indicate that if a dwellings's area is 2500 sq. ft. and the appraised building value is 150k dollars, the sale price will be about 167,500 dollars. Similarly, if the sq. ft. is 4500 and appraised value is 300000, respectively, the sale price is predicted to be about 321,900 dollars. However, we do need to remember that this simple regression (which does exclude many of the properties in our full dataset) is not particularly accurate.

<a name="Linear-Like-Regression"></a>
### Linear-Like Regression

While there are many of relationships that are linear and that can be modeled using a linear regression, there are also relationships that are non-linear.  Among these non-linear relationships are those that can transformed into a linear ones in terms of the coefficients and independent variables.

As an example, consider the relationship between `comb08` and `c02`.

In [ ]:
df_avg_mod2 = df_avg_mod[df_avg_mod.area < 10000]
df_avg_mod2 = df_avg_mod2[df_avg_mod2.area > 200]
df_avg_mod2.plot.scatter(x="yearbuilt", y="area", c='b')

While this relationship doesn't appear to be linear, it does look [hyperbolic](https://en.wikipedia.org/wiki/Hyperbola). In this case, the relationship between the independent variable and dependent variable could be written as 


$$
\begin{array}{c}
Y = \frac{1}{\beta_0 + \beta_1 X}
\end{array}
$$

To move the coefficients and independent variable out of the denominator, we can take the reciprocal of both sides (provided the neither side is zero) - this gives us


$$
\begin{array}{c}
\frac{1}{Y} = \beta_0 + \beta_1 X
\end{array}
$$

For a given observation, this form is easier to work with since $\frac{1}{Y}$ and $X$ are constants and we need to solve for $\beta_0$ and $\beta_1$. 

We can calculate the reciprocal of the dependent variable, `co2`, and store the value in a new column.

In [ ]:
df_avg_non_linear = df_avg_mod2[['yearbuilt', 'area']].copy()

To transform the problem into a linear one, we need to calculate the reciprocal of the `co2` values.

In [ ]:
df_avg_non_linear["reciprocal_area"] = 1 / df_avg_non_linear.area

Looking at the plot of `yearbuilt` and `area`, it appears that the relationship is linear, which supports our assumption that the original relationship was hyperbolic.

<hr>

<a name="Exercise-5"></a><mark> **Exercise 5** In the cell below, create a scatter plot of `yearbuilt` and `reciprocal_area`.</mark>

<hr> 

Before fitting the data with a linear model, let's remove the outliers.

<hr>

<a name="Exercise-6"></a><mark> **Exercise 6** In the cell below, remove the outliers from the `comb08` and `reciprocal_co2` columns in the `epa_non_linear` DataFrame. Use the *remove_outliers()* function we created earlier.</mark>

<hr>

With the outliers removed, let's look at the scatter plot again.

In [ ]:
axes = df_avg_non_linear.plot.scatter(x="yearbuilt", y="reciprocal_area", c='b')

We can now create a linear model for `comb08` and `reciprocal_c02`.  After calculating the coefficients, we display the summary of the results.

In [ ]:
X = sm.add_constant(df_avg_non_linear["area"])
Y = df_avg_non_linear["reciprocal_area"]
model = sm.OLS(Y, X)
res = model.fit()
display(res.summary())

From the coefficient of determination, we see that the model produced a good fit. Adding the regression line to the existing scatter plot give the following plot.

In [ ]:
abline_plot(model_results=res, ax=axes, color='r')

We now need to transform the regression line to fit the original data.  Using the *params* property of the results we have the following constant term and coefficient.

In [ ]:
res.params

This means that our model is 

$$
\begin{array}{c}
Y = \frac{1}{0.000068 + 0.000109 X}
\end{array}
$$

We can calculate the predicted values from our model using the following code.  We use *sort_values()* to ensure that the values of the independent variable are in order. This will be important when we plot the curve; we didn't need to do this previously as we relied on the *abline_plot()* function, which handled this for us,

In [ ]:
prediction = 1/(res.params.const + res.params.area * df_avg_mod2.yearbuilt.sort_values())

We can now plot the model curve against our original data. 

In [ ]:
fig, axes = plt.subplots(figsize=(10,10))
axes.plot(df_avg_mod2.yearbuilt, df_avg_mod2.area, 'b+')
axes.plot(df_avg_mod2.yearbuilt.sort_values(), prediction, 'r')

<a name="Logistic-Regression"></a>
## Logistic Regression

A [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) is used to model data where the dependent variable is categorical. In the simplest case, the dependent variable is binary and has only two possible values. A logistic model, provides an estimate of the probability that one of the two categories applies given the values of the independent variables; it fits a [logistic probability distribution](https://en.wikipedia.org/wiki/Logistic_distribution) to the data. We'll only look at simple case where the dependent variable is binary and there is only one independent variable.

### An Example 

As an example, consider the the following [example taken from the Wikipedia page on logistic regressions](https://en.wikipedia.org/wiki/Logistic_regression#Example:_Probability_of_passing_an_exam_versus_hours_of_study).  We have two variables: *hours* and *passed*.  The *hours* variable represents the number of hours a student spent studying for an exam and *passed* indicates whether or not the student passed the exam where `0` indicates failure and `1` indicates that the student passed; *hours* is a continuous variable and *passed* is a discrete, binary variable.

In [ ]:
hours = [0.5, 0.75, 1.0, 1.25, 1.50, 1.75, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 4.0, 4.25, 4.5, 4.75, 5.0, 5.5]
passed = [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]

Plotting this data as a scatter plot give the following.

In [ ]:
plt.plot(hours, passed, 'b+')

The logistic regression calculates values for $\beta_0$ and $\beta_1$ in the following formula

$$
\begin{array}{c}
P = \frac{1}{1 + e^{-(\beta_0 + \beta_1 X)}}
\end{array}
$$

where $P$ is is a probability value between 0 and 1 and $X$ is the independent variable.

We can use the StatsModels [*Logit()*](http://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.Logit.html) function to perform the logistic regression.  

We have to reassign the value of `stats.chisqprob` due to a discrepancy between the StatsModels module and the libraries on which it depends.

In [ ]:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

X = sm.add_constant(hours)
logit_model=sm.Logit(passed,X)
result=logit_model.fit()
display(result.summary())

We create the model in much the same way as we did for a linear model.  We specify the independent variable and add a constant using StatsModels' *add_constant()* function.  We next create a logistic model using the *Logit()* function.  To calculate the coefficients, we use model's *fit()* method.  After the calculation is complete, we can view a summary of the results.

Just like the linear model, the results of the logistic model have a *predict()* method that give the model's predicted values based on the values of the independent variable.  We can use this to plot the logistic curve along with the scatter plot of the data. 

The model represents the probability of passing the exam given some number of hours spent studying.

In [ ]:
fig, axes = plt.subplots(figsize=(10,8))
axes.plot(hours,passed, 'b+')
axes.plot(hours, result.predict(), 'r-')

We can calculate the probability of for a given value of the independent variable using a function.

In [ ]:
def probability(model_results, value):
    const, coeff = model_results.params
    exponent = -(const + coeff * value)
    denominator = 1 + np.exp(exponent)
    return 1 / denominator

print(probability(result, 2))
print(probability(result, 6))

For this problem, the results indicate if a student spends 2 hours studying there is about a 26% likelihood that the student will pass the exam; similarly if the student spends 6 hours studying, the probability of passing is greater than 99%.

### Home Data

For an example with real data, consider the count auditor data we worked with previously.  We can load the data from our local database.

For this example, lets see if we can calculate the logistic model that gives the probability of a house having a fireplace given its area.  Currently, the dataset includes the the number of fireplaces a given property has; we need to convert values greater than zero to 1, indicating that there is a fireplace.  

First, we drop any rows with missing data. Next, we create a new column, `HasFireplace` that is equal to the mask corresponding the `Fireplaces` being greater than zero. Masks return values of `True` or `False` and the *astype(int)* function call will convert the boolean value to an integer where `False` becomes 0 and `True` becomes 1. 

In [ ]:
df_avg['fireplacesflag'] = (df_avg.fireplacesflag > 0).astype(int)
df_avg.head(10)

We can now create the scatter plot of `Area` and `HasFireplaces`

In [ ]:
#df_avg_mod2.plot.scatter(x="area", y="fireplacesflag", c='b')
plt.plot(df_avg_mod2.area, df_avg_mod2.fireplacesflag, 'b+')

Before creating the model, we sort the values of our independent variable; this will aid in plotting later.

In [ ]:
df_avg_mod2.sort_values(by=["area"], inplace=True)

Creating the logistic model and calculating the regression coefficients is similar to the logistic model as we noted in the example.

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(df_avg_mod2.area)
logit_model=sm.Logit(df_avg_mod2.fireplacesflag,X)
result=logit_model.fit()
display(result.summary())

With the model created and the coefficients calculated, we can now plot the regression curve against the original data.

<hr>

<a name="Exercise-7"></a><mark> **Exercise 7** In the cell below, create a scatter plot of the `area` and `fireplacesflag` columns from the `df_avg_mod2` DataFrame. On the same figure, plot the logistic regression curve.</mark>

<hr>

The formula for the curve is given by

$$
\begin{array}{c}
P = \frac{1}{1 + e^{-(-1.7215 + 0.0008 X)}}
\end{array}
$$

For a given area, we can calculate the probability that the house has a fireplace using this formula based on the model.

Let's look at one more example.  Older home tend to have fewer bathrooms. We can create a new column, `MoreThanOneBathroom`

<hr>

<a name="Exercise-8"></a><mark> **Exercise 8** In the cell below, create a new column named `MoreThanOneBathroom` in the `df_avg` DataFrame that has a value of 0 if the value of `bathrooms` is less than or equal to 1 and has a value of 1 otherwise.</mark>

<hr>

We should sort the data by `yearbuilt` before continuing.

In [ ]:
df_avg.sort_values(by=["yearbuilt"], inplace=True)

In [ ]:
df_avg.MoreThanOneBathroom.isna().describe()

As before we construct the model with `yearbuilt` as the independent variable and `MoreThanOneBathroom` as the dependent variable. We calculate a logistic curve that best fits the data.

In [ ]:
X = sm.add_constant(df_avg.yearbuilt)
logit_model=sm.Logit(df_avg.MoreThanOneBathroom, X)
result=logit_model.fit()
display(result.summary())

Finally, we can create a scatter plot of `yearbuilt` and `MoreThanOneBathroom` along with the logistic regression curve.

In [ ]:
fig, axes = plt.subplots(figsize=(10,8))
axes.plot(df_avg.yearbuilt, df_avg.MoreThanOneBathroom, 'b+')
axes.plot(df_avg.yearbuilt, result.predict(), 'r-')

## Exercise Answers

1. ```python
   columns = [ "yearbuilt", "appraisedtaxablebuilding", "saleprice", "area", "bathrooms", "bedrooms", "acreage" ]
   sns.pairplot(df_avg[columns])
   ```
   
   
2. ```python
   df_avg.plot.scatter(x="area", y="bathrooms", c='b')
   ```
   
   
4. ```python
   X = sm.add_constant(epa_no_outliers.cylinders)
   Y = epa_no_outliers.displ
   model = sm.OLS(Y, X)
   res_no_outliers = model.fit()
   ```
   
   
5. ```python
   df_avg_non_linear.plot.scatter(x="yearbuilt", y="reciprocal_area", c='b')
   ```
   
   
6. ```python
   def remove_outliers(dataframe, column):
   q1 = dataframe[column].quantile(0.25)
   q3 = dataframe[column].quantile(0.75)
   iqr = q3 - q1
   lower = q1 - 1.5 * iqr
   upper = q3 + 1.5 * iqr
   return dataframe[(dataframe[column] >= lower) &
                    (dataframe[column] <= upper)].copy()
   df_avg_non_linear = remove_outliers(df_avg_non_linear, "yearbuilt")
   df_avg_non_linear = remove_outliers(df_avg_non_linear, "reciprocal_area")
   ```
   
   
7. ```python
   import matplotlib.pyplot as plt
   fig, axes = plt.subplots(figsize=(10,8))
   axes.plot(df_avg_mod2.area,df_avg_mod2.fireplacesflag, 'b+')
   axes.plot(df_avg_mod2.area, result.predict(), 'r-')
   ```


8. ```python
   df_avg["MoreThanOneBathroom"] = (df_avg.bathrooms > 1).astype(int)
   ```

## Next Steps

The models we create can be used populate dashboards or included in reports. Later, we'll look at creating visualizations and reporting information. Model creation could also be an intermediate step in the analysis process; in a later unit we'll look at automating model creation.

## Resources and Further Reading

- [Simple and Multiple Linear Regression in Python](https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9)
- [Logistic Regression in Python Using Rodeo](http://blog.yhat.com/posts/logistic-regression-python-rodeo.html)
- [Regression Analysis with Python by Massaron and Boschetti (Safari Books)](http://proquest.safaribooksonline.com.cscc.ohionet.org/book/programming/python/9781785286315)
- [Data Science Algorithms in a Week by Natingga, Regression (Safari Books)](http://proquest.safaribooksonline.com.cscc.ohionet.org/book/programming/machine-learning/9781787284586/regression/1500cb6b_9703_4b4a_bffb_61da8fbd2e97_xhtml?uicode=ohlink)

## Notes

1. <a name="note-1"></a> The null hypothesis being tested is that the variable associated with the coefficient has no effect on the dependent variable.  When the p-value is sufficiently low, typically less the 0.05, we reject the null hypothesis thereby accepting that the variable does have an effect on the dependent variable.